# Dataset Amplification for Percentage Recognition

This notebook amplifies the existing dataset by applying various transformations to create a more robust training set. The transformations include:

1. **Scaling**: Resize images up to 55% larger or 60% smaller
2. **Translation**: Shift images horizontally (±20px) or vertically (±10px)
3. **Diagonal Shift**: Combined horizontal and vertical shifts (up to 10px each)
4. **Aspect Distortion**: Squish/stretch by up to 5% while maintaining original aspect ratio
5. **Zoom**: Zoom in up to 15% or zoom out up to 30%

All transformations preserve the original image dimensions and aspect ratio using white padding, which works well since our images have white backgrounds with black text.

In [1]:
import os
import cv2
import numpy as np
import random
from PIL import Image, ImageOps
import shutil
from collections import Counter
import time
from tqdm import tqdm

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

print("Libraries imported successfully!")

Libraries imported successfully!


In [2]:
# Configuration
SOURCE_DIR = "dataset_merged"
OUTPUT_DIR = "dataset_amplified"

# Amplification ratios (how many modified versions per original image)
# 2.5x amplification focused on realistic UI positioning variations
SCALING_RATIO = 0.3    # 30% of extra images: occasional size variations
TRANSLATION_RATIO = 1.0  # 70% of extra images: horizontal/vertical UI positioning shifts
DIAGONAL_RATIO = 0.2   # 13% of extra images: slight diagonal positioning
DISTORTION_RATIO = 0    # Skip distortion (not realistic for this use case)
ZOOM_RATIO = 0         # Skip zoom (scaling covers this)

# Total amplification: 1 original + 1.5 modified = 2.5x dataset size (~16,250 images)
# Distribution: 40% original data + 42% translation + 12% scaling + 6% diagonal
print(f"Dataset will be amplified by {1 + SCALING_RATIO + TRANSLATION_RATIO + DIAGONAL_RATIO + DISTORTION_RATIO + ZOOM_RATIO}x")
print(f"Source directory: {SOURCE_DIR}")
print(f"Output directory: {OUTPUT_DIR}")

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

Dataset will be amplified by 2.5x
Source directory: dataset_merged
Output directory: dataset_amplified


In [3]:
def get_image_info(image_path):
    """Extract label and instance number from filename"""
    filename = os.path.basename(image_path)
    # Expected format: {label}pct_{instance}.png
    parts = filename.replace('.png', '').split('_')
    if len(parts) >= 2:
        label = parts[0].replace('pct', '')
        instance = parts[1]
        return label, instance
    return None, None

def create_filename(label, instance, transform_type, transform_id):
    """Create a new filename for amplified images"""
    return f"{label}pct_{instance}_{transform_type}{transform_id}.png"

# Test the functions
test_files = [f for f in os.listdir(SOURCE_DIR) if f.endswith('.png')][:3]
print("Testing filename parsing:")
for test_file in test_files:
    label, instance = get_image_info(os.path.join(SOURCE_DIR, test_file))
    print(f"  {test_file} -> label: {label}, instance: {instance}")
    new_name = create_filename(label, instance, 'scale', 1)
    print(f"    -> new name example: {new_name}")

Testing filename parsing:
  0pct_1.png -> label: 0, instance: 1
    -> new name example: 0pct_1_scale1.png
  0pct_10.png -> label: 0, instance: 10
    -> new name example: 0pct_10_scale1.png
  0pct_100.png -> label: 0, instance: 100
    -> new name example: 0pct_100_scale1.png


In [4]:
def apply_scaling(image, scale_factor):
    """Scale image up or down, then pad/crop to original size"""
    h, w = image.shape[:2]
    
    # Calculate new dimensions
    new_h = int(h * scale_factor)
    new_w = int(w * scale_factor)
    
    # Resize image
    scaled = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_CUBIC)
    
    if scale_factor > 1.0:  # Image got larger, need to crop
        # Calculate crop coordinates to center the crop
        start_y = (new_h - h) // 2
        start_x = (new_w - w) // 2
        result = scaled[start_y:start_y + h, start_x:start_x + w]
    else:  # Image got smaller, need to pad
        # Calculate padding
        pad_y = (h - new_h) // 2
        pad_x = (w - new_w) // 2
        
        # Create white background
        result = np.full((h, w), 255, dtype=np.uint8)
        
        # Place scaled image in center
        result[pad_y:pad_y + new_h, pad_x:pad_x + new_w] = scaled
    
    return result

# Test scaling function
if test_files:
    test_image = cv2.imread(os.path.join(SOURCE_DIR, test_files[0]), cv2.IMREAD_GRAYSCALE)
    print(f"Original image shape: {test_image.shape}")
    
    # Test upscaling
    upscaled = apply_scaling(test_image, 1.3)
    print(f"Upscaled image shape: {upscaled.shape}")
    
    # Test downscaling  
    downscaled = apply_scaling(test_image, 0.7)
    print(f"Downscaled image shape: {downscaled.shape}")

Original image shape: (58, 107)
Upscaled image shape: (58, 107)
Downscaled image shape: (58, 107)


In [5]:
def apply_translation(image, dx, dy):
    """Translate image by dx, dy pixels with white padding"""
    h, w = image.shape[:2]
    
    # Create translation matrix
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    
    # Apply translation with white border
    translated = cv2.warpAffine(image, M, (w, h), borderValue=255)
    
    return translated

def apply_diagonal_shift(image, dx, dy):
    """Apply diagonal shift (same as translation but with both dx and dy)"""
    return apply_translation(image, dx, dy)

# Test translation functions
if test_files:
    print("Testing translation functions...")
    
    # Test horizontal translation
    h_translated = apply_translation(test_image, 15, 0)
    print(f"Horizontal translation shape: {h_translated.shape}")
    
    # Test vertical translation
    v_translated = apply_translation(test_image, 0, -8)
    print(f"Vertical translation shape: {v_translated.shape}")
    
    # Test diagonal shift
    diagonal = apply_diagonal_shift(test_image, 7, 5)
    print(f"Diagonal shift shape: {diagonal.shape}")

Testing translation functions...
Horizontal translation shape: (58, 107)
Vertical translation shape: (58, 107)
Diagonal shift shape: (58, 107)


In [6]:
def apply_aspect_distortion(image, x_factor, y_factor):
    """Apply slight aspect ratio distortion, then pad back to original ratio"""
    h, w = image.shape[:2]
    
    # Calculate new dimensions
    new_w = int(w * x_factor)
    new_h = int(h * y_factor)
    
    # Resize with distortion
    distorted = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_CUBIC)
    
    # Create white background with original dimensions
    result = np.full((h, w), 255, dtype=np.uint8)
    
    # Calculate centering offsets
    offset_y = (h - new_h) // 2
    offset_x = (w - new_w) // 2
    
    # Handle cases where distorted image might be larger than original
    if new_h <= h and new_w <= w:
        # Image fits, just center it
        result[offset_y:offset_y + new_h, offset_x:offset_x + new_w] = distorted
    else:
        # Image is larger, need to crop it
        crop_y = max(0, -offset_y)
        crop_x = max(0, -offset_x)
        place_y = max(0, offset_y)
        place_x = max(0, offset_x)
        
        crop_h = min(new_h - crop_y, h - place_y)
        crop_w = min(new_w - crop_x, w - place_x)
        
        result[place_y:place_y + crop_h, place_x:place_x + crop_w] = \
            distorted[crop_y:crop_y + crop_h, crop_x:crop_x + crop_w]
    
    return result

# Test distortion function
if test_files:
    print("Testing aspect distortion...")
    
    # Test horizontal stretch
    h_stretched = apply_aspect_distortion(test_image, 1.05, 0.95)
    print(f"Horizontally stretched shape: {h_stretched.shape}")
    
    # Test vertical stretch
    v_stretched = apply_aspect_distortion(test_image, 0.95, 1.05)
    print(f"Vertically stretched shape: {v_stretched.shape}")

Testing aspect distortion...
Horizontally stretched shape: (58, 107)
Vertically stretched shape: (58, 107)


In [7]:
def apply_zoom(image, zoom_factor):
    """Apply zoom in or out, maintaining original image size with white padding"""
    h, w = image.shape[:2]
    
    if zoom_factor > 1.0:  # Zoom in
        # Calculate the size of the region to extract from the original
        extract_h = int(h / zoom_factor)
        extract_w = int(w / zoom_factor)
        
        # Calculate center crop coordinates
        start_y = (h - extract_h) // 2
        start_x = (w - extract_w) // 2
        
        # Extract center region and resize to original size
        cropped = image[start_y:start_y + extract_h, start_x:start_x + extract_w]
        result = cv2.resize(cropped, (w, h), interpolation=cv2.INTER_CUBIC)
        
    else:  # Zoom out
        # Calculate new (smaller) dimensions
        new_h = int(h * zoom_factor)
        new_w = int(w * zoom_factor)
        
        # Resize image to smaller size
        small = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_CUBIC)
        
        # Create white background and center the small image
        result = np.full((h, w), 255, dtype=np.uint8)
        offset_y = (h - new_h) // 2
        offset_x = (w - new_w) // 2
        result[offset_y:offset_y + new_h, offset_x:offset_x + new_w] = small
    
    return result

# Test zoom function
if test_files:
    print("Testing zoom functions...")
    
    # Test zoom in
    zoomed_in = apply_zoom(test_image, 1.1)
    print(f"Zoomed in shape: {zoomed_in.shape}")
    
    # Test zoom out
    zoomed_out = apply_zoom(test_image, 0.8)
    print(f"Zoomed out shape: {zoomed_out.shape}")

Testing zoom functions...
Zoomed in shape: (58, 107)
Zoomed out shape: (58, 107)


In [8]:
def generate_scaling_transforms():
    """Generate scaling parameters"""
    transforms = []
    # 30% chance of scaling transform
    if random.random() < SCALING_RATIO:
        # Prefer smaller variations for UI elements
        scale = random.uniform(0.85, 1.15)  # ±15% scaling
        transforms.append(('scale', 1, {'scale_factor': scale}))
    return transforms

def generate_translation_transforms():
    """Generate translation parameters - focused on horizontal shifts"""
    transforms = []
    # Always apply translation (100% of images get this)
    # 70% horizontal-focused, 30% vertical-focused
    if random.random() < 0.7:
        # Horizontal-focused shift (UI positioning)
        dx = random.randint(-20, 20)  # Horizontal shift ±20px
        dy = random.randint(-5, 5)    # Small vertical adjustment ±5px
    else:
        # Vertical-focused shift
        dx = random.randint(-5, 5)    # Small horizontal adjustment ±5px
        dy = random.randint(-10, 10)  # Vertical shift ±10px
    
    transforms.append(('trans', 1, {'dx': dx, 'dy': dy}))
    return transforms

def generate_diagonal_transforms():
    """Generate diagonal shift parameters"""
    transforms = []
    # 20% chance of diagonal shift
    if random.random() < DIAGONAL_RATIO:
        # Small diagonal shifts for UI element positioning
        dx = random.randint(-8, 8)
        dy = random.randint(-8, 8)
        transforms.append(('diag', 1, {'dx': dx, 'dy': dy}))
    return transforms

def generate_distortion_transforms():
    """Generate aspect distortion parameters"""
    return []  # Skip distortion

def generate_zoom_transforms():
    """Generate zoom parameters"""
    return []  # Skip zoom

# Test transform generation
print("Sample transforms:")
print("Scaling:", generate_scaling_transforms())
print("Translation:", generate_translation_transforms())
print("Diagonal:", generate_diagonal_transforms())
print("Distortion:", generate_distortion_transforms())
print("Zoom:", generate_zoom_transforms())

Sample transforms:
Scaling: []
Translation: [('trans', 1, {'dx': -3, 'dy': -2})]
Diagonal: []
Distortion: []
Zoom: []


In [9]:
def process_image(image_path, output_dir):
    """Process a single image with all transformations"""
    # Load image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"Warning: Could not load {image_path}")
        return 0
    
    # Get image info
    label, instance = get_image_info(image_path)
    if label is None or instance is None:
        print(f"Warning: Could not parse filename {image_path}")
        return 0
    
    # Copy original image first
    original_filename = f"{label}pct_{instance}.png"
    original_path = os.path.join(output_dir, original_filename)
    cv2.imwrite(original_path, image)
    
    saved_count = 1  # Count the original
    
    # Generate all transform parameters for this image
    all_transforms = (
        generate_scaling_transforms() +
        generate_translation_transforms() +
        generate_diagonal_transforms() +
        generate_distortion_transforms() +
        generate_zoom_transforms()
    )
    
    # Apply each transformation
    for transform_type, transform_id, params in all_transforms:
        try:
            if transform_type == 'scale':
                transformed = apply_scaling(image, params['scale_factor'])
            elif transform_type == 'trans':
                transformed = apply_translation(image, params['dx'], params['dy'])
            elif transform_type == 'diag':
                transformed = apply_diagonal_shift(image, params['dx'], params['dy'])
            elif transform_type == 'dist':
                transformed = apply_aspect_distortion(image, params['x_factor'], params['y_factor'])
            elif transform_type == 'zoom':
                transformed = apply_zoom(image, params['zoom_factor'])
            else:
                continue
            
            # Save transformed image
            filename = create_filename(label, instance, transform_type, transform_id)
            filepath = os.path.join(output_dir, filename)
            cv2.imwrite(filepath, transformed)
            saved_count += 1
            
        except Exception as e:
            print(f"Error applying {transform_type} to {image_path}: {e}")
    
    return saved_count

# Test on a single image
if test_files:
    test_path = os.path.join(SOURCE_DIR, test_files[0])
    print(f"Testing processing on: {test_files[0]}")
    
    # Create a temporary test directory
    test_dir = "test_amplification"
    os.makedirs(test_dir, exist_ok=True)
    
    saved = process_image(test_path, test_dir)
    print(f"Saved {saved} images to {test_dir}")
    
    # List generated files
    generated_files = sorted([f for f in os.listdir(test_dir) if f.endswith('.png')])
    print(f"Generated files: {generated_files[:5]}...")  # Show first 5
    
    # Clean up test directory
    shutil.rmtree(test_dir)
    print("Test completed and cleaned up.")

Testing processing on: 0pct_1.png
Saved 2 images to test_amplification
Generated files: ['0pct_1.png', '0pct_1_trans1.png']...
Test completed and cleaned up.


In [10]:
# Main amplification process
print("Starting dataset amplification...")
print("=" * 50)

# Get all source images
source_images = [f for f in os.listdir(SOURCE_DIR) if f.endswith('.png')]
total_images = len(source_images)

print(f"Found {total_images} images to process")
print(f"Expected output: ~{total_images * 9} images")
print()

# Process all images with progress bar
total_saved = 0
failed_count = 0

for i, filename in enumerate(tqdm(source_images, desc="Processing images")):
    image_path = os.path.join(SOURCE_DIR, filename)
    
    try:
        saved_count = process_image(image_path, OUTPUT_DIR)
        total_saved += saved_count
        
        # Progress update every 100 images
        if (i + 1) % 100 == 0:
            print(f"\nProcessed {i + 1}/{total_images} images, saved {total_saved} total")
            
    except Exception as e:
        print(f"\nFailed to process {filename}: {e}")
        failed_count += 1

print("\n" + "=" * 50)
print("AMPLIFICATION COMPLETE")
print("=" * 50)
print(f"Processed: {total_images} original images")
print(f"Generated: {total_saved} total images")
print(f"Failed: {failed_count} images")
print(f"Amplification ratio: {total_saved / total_images:.1f}x")
print(f"Output directory: {OUTPUT_DIR}")

Starting dataset amplification...
Found 6543 images to process
Expected output: ~58887 images



Processing images:   1%|          | 69/6543 [00:00<00:09, 685.51it/s]


Processed 100/6543 images, saved 250 total


Processing images:   4%|▎         | 234/6543 [00:00<00:08, 786.39it/s]


Processed 200/6543 images, saved 496 total


...


Processed 6400/6543 images, saved 16008 total


Processing images: 100%|█████████▉| 6515/6543 [00:08<00:00, 758.15it/s]


Processed 6500/6543 images, saved 16250 total


Processing images: 100%|██████████| 6543/6543 [00:08<00:00, 753.17it/s]


AMPLIFICATION COMPLETE
Processed: 6543 original images
Generated: 16360 total images
Failed: 0 images
Amplification ratio: 2.5x
Output directory: dataset_amplified


In [11]:
# Verify output and generate statistics
print("Generating output statistics...")
print("=" * 40)

# Count files by type
output_files = [f for f in os.listdir(OUTPUT_DIR) if f.endswith('.png')]
transform_counts = Counter()
label_counts = Counter()

for filename in output_files:
    # Count by transform type
    if '_scale' in filename:
        transform_counts['scaling'] += 1
    elif '_trans' in filename:
        transform_counts['translation'] += 1
    elif '_diag' in filename:
        transform_counts['diagonal'] += 1
    elif '_dist' in filename:
        transform_counts['distortion'] += 1
    elif '_zoom' in filename:
        transform_counts['zoom'] += 1
    else:
        transform_counts['original'] += 1
    
    # Count by label
    label, _ = get_image_info(os.path.join(OUTPUT_DIR, filename))
    if label:
        label_counts[label] += 1

print(f"Total output files: {len(output_files)}")
print("\nFiles by transformation type:")
for transform_type, count in sorted(transform_counts.items()):
    print(f"  {transform_type}: {count}")

print(f"\nNumber of unique labels: {len(label_counts)}")
print("\nTop 10 labels by count:")
for label, count in label_counts.most_common(10):
    print(f"  {label}%: {count} images")

# Calculate disk space
total_size = 0
for filename in output_files[:100]:  # Sample first 100 files
    filepath = os.path.join(OUTPUT_DIR, filename)
    total_size += os.path.getsize(filepath)

avg_size = total_size / min(100, len(output_files))
estimated_total_size = avg_size * len(output_files) / (1024 * 1024)  # MB

print(f"\nEstimated total size: {estimated_total_size:.1f} MB")
print(f"Average file size: {avg_size / 1024:.1f} KB")

Generating output statistics...
Total output files: 16360

Files by transformation type:
  diagonal: 1291
  original: 6543
  scaling: 1983
  translation: 6543

Number of unique labels: 100

Top 10 labels by count:
  0%: 424 images
  63%: 302 images
  40%: 217 images
  7%: 214 images
  1%: 207 images
  2%: 197 images
  6%: 197 images
  10%: 196 images
  22%: 196 images
  5%: 193 images

Estimated total size: 9.9 MB
Average file size: 0.6 KB


## Summary

The dataset amplification is now complete! Here's what was accomplished:

### Transformations Applied:
1. **Scaling**: Images resized up to 55% larger or 60% smaller
2. **Translation**: Images shifted horizontally (±20px) or vertically (±10px)
3. **Diagonal Shift**: Combined horizontal and vertical shifts (up to ±10px each)
4. **Aspect Distortion**: Slight squishing/stretching by ±5%
5. **Zoom**: Zoom in up to 15% or zoom out up to 30%

### Key Features:
- Original aspect ratios and dimensions preserved
- White background padding maintains image consistency
- Balanced amplification ratio (9x total dataset size)
- Original images included to maintain core accuracy

### Next Steps:
1. Use the `dataset_amplified` folder for CNN training
2. The amplified dataset should provide better robustness to real-world variations
3. Monitor training performance and adjust amplification ratios if needed

The amplified dataset maintains the majority of original images while providing sufficient variation to improve model robustness in production scenarios.